In [1]:
from helper import *
from Simulation.ForwardKinematics import ForwardKinematics
from Simulation.DifferentialKinematics import DifferentialKinematics

from neura_dual_quaternions import DualQuaternion, Quaternion
from Simulation.LinearInterpolator import LinearInterpolator

import roboticstoolbox as rtb
import spatialmath as sm
import spatialgeometry as sg
import numpy as np
from swift import Swift

import time
np.set_printoptions(precision=6, suppress=True, linewidth=1200, threshold=np.inf)

In [56]:
x1 = DualQuaternion.fromQuatPos(Quaternion.fromAxisAngle(np.pi*0.5, np.array([0,1,0])), np.array([0.815,0,1.138]))
x2 = DualQuaternion.fromQuatPos(Quaternion.fromAxisAngle(np.pi*0.5, np.array([0,1,0])), np.array([0.815,0.2,1.138]))   

q0 = np.array([np.pi*0.5,np.pi*0.5,0,0,0,0,0])


J = fk.jacobian_body(q0)


# Compute the SVD
U, S, Vt = np.linalg.svd(J)

cartesian_direction = np.array([0,0,0, 0,0,0])  # Replace x, y, z with your direction components

# Normalize the direction vector
cartesian_direction = cartesian_direction / np.linalg.norm(cartesian_direction)

# Project the Cartesian direction onto the left singular vectors and multiply by singular values
manipulability_in_direction = np.dot(U*S,  cartesian_direction)
print(np.linalg.norm(manipulability_in_direction))
print(dk.dir_manipulability(q0, cartesian_direction))

2.330531199593773
1.815275461190397


In [24]:
q1 = np.array([-0.677,  0.208,  0.791,  1.547,  0.388, -0.141, -0.531])

print(dk.dir_manipulability_gradient3(q1))

print(dk.manipulability(q1))

[-0.        0.308913 -0.563009 -0.006197  0.294582  0.44316  -0.026044]
0.10185434494555715


In [4]:
q1 = np.array([-0.677,  0.208,  0.791,  1.547,  0.388, -0.141, -0.531])
q2 = np.array([-0.687,  0.21,   0.804,  1.547,  0.393, -0.142, -0.54])
q3 = (q1 + q2)*0.5
v = [1, 0, 0, 0, 1, 0]
print(dk.dir_manipulability(q1, v))
print(dk.dir_manipulability(q3, v))
print(dk.dir_manipulability(q2, v))

print(dk.dir_manipulability_gradient(q1, v))
print(dk.dir_manipulability_gradient(q3, v))
print(dk.dir_manipulability_gradient(q2, v))

Jb = fk.jacobian_body(q1)
U, S, Vh = np.linalg.svd(Jb@Jb.T)
print(U)
print(S)

Jb = fk.jacobian_body(q3)
U, S, Vh = np.linalg.svd(Jb@Jb.T)
print(U)
print(S)

Jb = fk.jacobian_body(q2)
U, S, Vh = np.linalg.svd(Jb@Jb.T)
print(U)
print(S)

1.6903383455160006
1.5173263542385491
1.5821155398748201
[ 0.       -6.668876  7.50717   4.017953 -2.83828   3.544385 -1.889526]
[  0.       -86.416601 -96.089278 -47.177078 -95.924938   4.83087   -1.779227]
[ -0.        30.92397   -1.49334   -1.994111  24.239465 -10.270669  -1.709053]
[[ 0.002858  0.767771  0.014095 -0.061606  0.415122  0.483942]
 [ 0.692729 -0.157546  0.404186  0.500268  0.014395  0.285419]
 [ 0.470483 -0.001957 -0.875844  0.070411  0.075415 -0.029891]
 [ 0.492441 -0.033193  0.241136 -0.726192  0.287896 -0.296671]
 [-0.196489 -0.619898 -0.096665 -0.177246  0.473892  0.558376]
 [ 0.13288   0.018102 -0.042904 -0.426858 -0.717162  0.532584]]
[3.767832 3.756263 2.100892 0.226468 0.190467 0.008089]
[[ 0.491415  0.590226 -0.014414  0.072163  0.415946  0.481371]
 [ 0.431243 -0.561918 -0.407549 -0.499077  0.028328  0.286882]
 [ 0.364733 -0.304235  0.87341  -0.068311  0.077504 -0.029887]
 [ 0.357298 -0.338784 -0.243833  0.7332    0.267819 -0.297993]
 [-0.546809 -0.351587  0.0

In [5]:
# env = Swift()
# robot = rtb.models.URDF.Maira7M()

# env.launch(realtime=True, browser="notebook")
# env.add(robot)

# # end-effector axes
# ee_axes = sg.Axes(0.1)

# env.add(ee_axes)
# np.set_printoptions(suppress=True, precision=3)

# # Change the robot configuration to the ready position
# robot.q = np.array([0, 0.0, 0, np.pi / 2, 0.00, -0.0, 0])
# robot.qd = np.zeros(robot.n)




# # Step the sim to view the robot in this configuration
# env.step(0)

# # Specify our desired end-effector velocity
# ev = [0, 0.0, 0.0, 0, 0.0, 0]
# ev2 = [1, 0.0, 0.0, 0.0, 0.0, 0.0]
# # Specify our timestep
# dt = 0.005


# # Run the simulation for 4 seconds
# for i in range(1000):

#     # Work out the manipulator Jacobian, Hessian using the
#     # current robot configuration
#     J = fk.jacobian_body(robot.q)
#     grad = dk.manipulability_gradient(robot.q)
#     grad2 = dk.dir_manipulability_gradient(robot.q, ev2)
#     print(dk.manipulability(robot.q), dk.dir_manipulability(robot.q, ev2))
#     # Calculate δq, the change to qd
#     pinv = np.linalg.pinv(J)
#     v = pinv @ ev + 20*(np.eye(7) - pinv@J)@grad2

#     #robot.qd = qd - δq
#     robot.q += v*dt
    
#     # Update the ee axes
#     ee_axes.T = fk.forward_kinematics(robot.q).asTransformation()

#     # Step the simulator by dt seconds
#     env.step(dt)